In [1]:
from textractor import Textractor

extractor = Textractor(profile_name="default")

In [21]:

import sys
import subprocess
import re


def convert_to(folder, source, timeout=None):
    args = [libreoffice_exec(), '--headless', '--convert-to', 'pdf', '--outdir', folder, source]

    process = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=timeout)
    filename = re.search('-> (.*?) using filter', process.stdout.decode())

    if filename is None: 
        raise LibreOfficeError(process.stdout.decode())
    else:
        return filename.group(1)


def libreoffice_exec():
    # TODO: Provide support for more platforms
    if sys.platform == 'darwin':
        return '/Applications/LibreOffice.app/Contents/MacOS/soffice'
    return 'libreoffice'


class LibreOfficeError(Exception):
    def __init__(self, output):
        self.output = output


if __name__ == '__main__':
    print('Converted to ' + convert_to(
        "./",
        "test_docs/112423 Google Drive Folder Test Outline v1 (1).docx"))

Converted to /home/cnadmin/official/myne/myne-ml-serve/research/112423 Google Drive Folder Test Outline v1 (1).pdf


In [1]:
import os 
os.chdir("..")

In [2]:
# from src.file_readers.readers import read_docx
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
    chunk_overlap=200, 
    separators=["\n\n", "\n", " ", ""])

# read_docx("research/test_docs/112423 Google Drive Folder Test Outline v1 (1).docx",
#          splitter)

In [3]:
from src.file_readers import ReadFiles
from src.entity.artifact_entity import FileHandlerArtifact
artifact = FileHandlerArtifact(
    file_storage_dir="artifacts/06_26_2024_12_48_34/file_storage/testmyne45"
)

In [4]:
rd = ReadFiles(file_handler_artifact=artifact,
               text_splitter=splitter)

In [5]:
files = rd.read_all_files()

In [11]:
files.keys()

dict_keys(['screenshot-(3).png', 'myne-project-plan.pdf'])

In [8]:
files.keys()

dict_keys(['screenshot-(3).png', 'myne-project-plan.pdf'])

In [7]:
import chromadb

chromadb.DEFAULT_DATABASE

'default_database'

In [2]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter
# from langchain_community.embeddings
import chromadb

persistent_client = chromadb.PersistentClient()
for doc_name, docs in files.items():
    
    collection = persistent_client.get_or_create_collection(doc_name.split(".")[0])
    collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


langchain_chroma = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embedding_function,
)

print("There are", langchain_chroma._collection.count(), "in the collection")

/home/cnadmin/official/myne/myne-ml-serve/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/cnadmin/official/myne/myne-ml-serve/env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/cnadmin/official/myne/myne-ml-serve/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0

There are 3 in the collection


In [9]:
from chromadb.api.types import Settings
import chromadb

client = chromadb.HttpClient(settings=Settings(
    chroma_db_impl="duckdb+s3",
    persist_directory="s3://ml-vectordb/chroma-data",
))


ImportError: cannot import name 'Settings' from 'chromadb.api.types' (/home/cnadmin/official/myne/myne-ml-serve/env/lib/python3.10/site-packages/chromadb/api/types.py)